In [7]:
import mysql.connector

conn = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "42krScm9AYbnucK.root",
  password = "aBNY8oPEWPSEo2YM",
  database = "vector_db",
  ssl_ca = "isrgrootx1.pem",
  ssl_verify_cert = True,
  ssl_verify_identity = True
)

In [9]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("BAAI/bge-m3")
text = "I love Machine Learning!"
embedding = embedder.encode(text).tolist()

In [11]:
len(embedding)

1024

In [5]:
documents = [
    "คณะวิทยาศาสตร์ มหาวิทยาลัยสงขลานครินทร์ วิทยาเขตหาดใหญ่ จัดตั้งขึ้นในปี พ.ศ. 2510 ซึ่งเป็นปีเดียวกับการก่อตั้งมหาวิทยาลัยสงขลานครินทร์",
    "คณะวิทยาศาสตร์ ประกอบด้วย 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์",
    "จัดการเรียนการสอนออกเป็นหลักสูตรปริญญาตรี 13 หลักสูตร และระดับบัณฑิตศึกษา 28 หลักสูตร คือปริญญาโท 17 หลักสูตร (มีหลักสูตรนานาชาติ 5 หลักสูตร) และปริญญาเอก 11 หลักสูตร (มีหลักสูตรนานาชาติ 6 หลักสูตร)",
    "วิสัยทัศน์ คือ คณะวิทยาศาสตร์เพื่อการพัฒนาที่ยั่งยืน",
    "คณบดี คือ ศาสตราจารย์ ดร.อัญชนา ประเทพ"
]

In [10]:
import json

def add_document(conn, text):
    cur = conn.cursor()
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (document, embedding) VALUES (%s , %s)", (text, json.dumps(embedding)))
    conn.commit()
    cur.close()

for doc in documents:
    add_document(conn, doc)

In [13]:
def query_tidb(conn, query_text, k=5):
    query_embedding = json.dumps(embedder.encode(query_text).tolist())
    
    cur = conn.cursor()

    sql_query = f"""
        SELECT document, vec_cosine_distance(embedding, '{query_embedding}') AS distance
        FROM documents
        ORDER BY distance
        LIMIT {k}
    """

    cur.execute(sql_query)
    results = cur.fetchall()
    cur.close()

    return [(content, distance) for content, distance in results]

results = query_tidb(conn, "คณะวิทย์มีสาขาอะไรบ้าง")
results

[('คณะวิทยาศาสตร์ ประกอบด้วย 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์',
  0.31030541246629073),
 ('คณะวิทยาศาสตร์ ประกอบด้วย 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์',
  0.31030541246629073),
 ('วิสัยทัศน์ คือ คณะวิทยาศาสตร์เพื่อการพัฒนาที่ยั่งยืน', 0.41161557851391073),
 ('วิสัยทัศน์ คือ คณะวิทยาศาสตร์เพื่อการพัฒนาที่ยั่งยืน', 0.41161557851391073),
 ('จัดการเรียนการสอนออกเป็นหลักสูตรปริญญาตรี 13 หลักสูตร และระดับบัณฑิตศึกษา 28 หลักสูตร คือปริญญาโท 17 หลักสูตร (มีหลักสูตรนานาชาติ 5 หลักสูตร) และปริญญาเอก 11 หลักสูตร (มีหลักสูตรนานาชาติ 6 หลักสูตร)',
  0.49062994178355)]

In [16]:
import ollama

def generate_response(conn, query_text):
    retrived_docs = query_tidb(conn, query_text)
    print(retrived_docs)
    context = "\n".join([doc[0] for doc in retrived_docs])
    print("---------")
    print(context)
    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query_text}"
    #prompt
    print("---------")
    print(prompt)
    response = ollama.chat(model="llama3.2", messages=[
        {"role": "system", "content": "คุณคือประธานสาขาเทคโนโลยีสารสนเทศและการสื่อสาร มหาวิทยาลัยสงขลานครินทร์"},
        {"role": "user", "content": prompt}
    ])
    #response
    print("---------")
    print(response)
    print(response["message"]["content"])
    return response["message"]["content"]

answer = generate_response(conn, "ภาคการศึกษาที่ 1 ต้องลงเรียนวิชาอะไรบ้าง")
print("---------")
answer

[('ม                                               19(x-y-z) \n \n \n \nป ี ที่ 2 \nภาคการศึกษาที่ 1 \n308-221 ทัศนศิลป ์ และการออกแบบ 2 2((1)-2-3) \n308-231 การโปรแกรมเชิงการค านวณและข้อมูลโครงสร้าง 3((2)-2-5) \n308-241 โครงสร้างระบบคอมพิวเตอร์และสถาปัตยกรรม 3((2)-2-5) \n315-201 ชีวิตแห่งอนาคต 2(2-0-4) \n315-202 การคิดกับการใช้เหตุผล 2(2-0-4) \n345-104 รู้ทันเทคโนโลยีดิจิทัล 2(2-0-4) \n315-100 ค านวณศิลป ์ 2((2)-0-4) \nxxx-xxx เลือกรายวิชากลุ่มสาระภาษาและการสื่อสาร 2(x-y-z) \n                               รวม ', 0.42915355302506397), ('8-121 ทัศนศิลป ์ และการออกแบบ 1 2((1)-2-3) \n308-131 ขั้นตอนวิธีและพื้นฐานการเขียนโปรแกรม 3((2)-2-5) \n001-103 ไอเดียสู่ความเป็นผู้ประกอบการ 1(1-0-2) \n895-100 พลเมืองที่ดี 2(x-y-z) \nxxx-xxx เลือกรายวิชากลุ่มสาระภาษาและการสื่อสาร 2(x-y-z) \nxxx-xxx เลือกรายวิชาเลือกกลุ่มสาระที่เปิ ดสอนในคณะ/วิทยาเขต 2(x-y-z) \nxxx-xxx เลือกรายวิชากลุ่มสาระสุนทรียศาสตร์และกีฬา 2(x-y-z) \n                               รวม                                               1

'จากข้อมูลที่ให้มา ภาคการศึกษาที่ 1 ควรลงเรียนรายวิชา 2((1)-2-3) ในกลุ่มการ-programming และชีวิตแห่งอนาคต'

In [15]:
answer

'จากตารางเรียนที่ให้มา เราจะเห็นว่าภาคการศึกษาที่ 1 มีรายชื่อวิชาที่กำหนดไว้ 13 หลักสูตร (รวมวิชาเลือกเสรี) และไม่มีวิชาอื่นใดที่ระบุไว้ในแบบที่ 1'

In [1]:
pip install pypdf


     ---------------------------------------- 0.0/330.6 kB ? eta -:--:--
     --- ----------------------------------- 30.7/330.6 kB 1.4 MB/s eta 0:00:01
     ---------- ---------------------------- 92.2/330.6 kB 1.1 MB/s eta 0:00:01
     ------------------------- ------------ 225.3/330.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 330.6/330.6 kB 1.9 MB/s eta 0:00:00


In [2]:
from pypdf import PdfReader

def read_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


In [3]:
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks


In [11]:
pdf_text = read_pdf("ict2564.pdf")
chunks = chunk_text(pdf_text)

for chunk in chunks:
    add_document(conn, chunk)